In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange

from utils.settings import config as cfg
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
J_tr, P_tr, P_ts, F = load_all_data(panda)
# Build Generative model, NSF
config = {
        'subnet_width': 1400,
        'subnet_num_layers': 3,
        'num_transforms': 9,
        'lr': 2.1e-4,
        'lr_weight_decay': 2.7e-2,
        'decay_step_size': 4e4,
        'gamma': 5e-2,
        'batch_size': 128,
        'num_epochs': 10,
    }
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model(
        enable_load_model=cfg.use_pretrained,
        num_transforms=config["num_transforms"],
        subnet_width=config["subnet_width"],
        subnet_num_layers=config["subnet_num_layers"],
        lr=config["lr"],
        lr_weight_decay=config["lr_weight_decay"],
        decay_step_size=config["decay_step_size"],
        gamma=config["gamma"],
        device='cuda')
knn = get_knn(P_tr=P_tr)

F load successfully from /home/luca/ikpflow/data/panda/train/F-2500000-7-7-1.npy
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth
knn load successfully from /home/luca/ikpflow/data/panda/train/knn-2500000-7-7-1-normFalse.pickle


In [3]:
# mu = torch.randn(size=(7,))
mu = torch.zeros(size=(cfg.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [4]:
# iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.01)
# test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=iflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [5]:
# nflow = get_nflow_model(flow=flow)
# test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=nflow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [6]:
# total_params = sum(
# 	param.numel() for param in flow.parameters()
# )
# print(total_params)

In [7]:
Path_dir = sample_P_path(robot=panda, load_time='0816002845')

/home/luca/ikpflow/data/panda/trajectory/0816002845/ load successfully.


In [9]:
def path_following(
    robot,
    Path_dir: str,
    model,
    knn,
    F,
    num_traj: int = 3,
) -> None:
    """
    path following generation for our method
    
    Parameters
    ----------
    robot : _type_
        robotic arm
    Path_dir : str
        path to ee Path, generated by sample_P_path
    model : _type_
        flow, iflow, or nflow
    knn : _type_
        knn of P_train
    F : _type_
        F_train
    num_traj : int, optional
        the number of generated joint trajectory samples, by default 3
    """

    def load_and_plot(exp_traj_path: str, ee_path: np.array):
        if os.path.exists(path=exp_traj_path):
            robot.plot(qs=qs)
        else:
            print(f"{exp_traj_path} does not exist !")

    Path = load_numpy(file_path=Path_dir + "ee_traj.npy")
    # Path = Path[:, :3]

    ref_F = nearest_neighbor_F(knn, np.atleast_2d(Path[0]), F, n_neighbors=40_0000) # knn
    # ref_F = F
    # ref_F = rand_F(Path[0], F)
    
    exp_path = lambda idx: Path_dir + f"exp_{idx}.npy"
    
    rand_idxs = np.random.randint(low=0, high=len(ref_F), size=num_traj)
    
    # rand_idxs = list(range(num_traj))
    for i, rand in enumerate(rand_idxs):
        # ref_F = rand_F(Path[0], F)
        
        # df, qs = sample_J_traj(Path, ref_F[rand], model, robot) 
        df, qs = sample_J_traj(Path, ref_F[rand], model, robot) 
        print("="*6 + str(rand) + f"=({ref_F[rand]})" + "="*6)
        print(df.describe())
        save_numpy(file_path=exp_path(i), arr=qs)

    for i in range(num_traj):
        load_and_plot(exp_traj_path=exp_path(i), ee_path=Path)

In [10]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0)
nflow = get_nflow_model(flow=flow, shrink_ratio=0.2)

In [12]:
path_following(robot=panda, Path_dir=Path_dir, model=iflow, knn=knn, F=F, num_traj=3)

======358083=([0.12083742])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.008655      23.594593
std           0.004306      17.352919
min           0.002421       0.418049
25%           0.005614       8.219036
50%           0.006798      24.395612
75%           0.012703      36.982781
max           0.015653      54.462841
======359783=([0.1912336])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.014673      99.511118
std           0.023193     167.050366
min           0.001143       0.532058
25%           0.006719       8.711460
50%           0.008205      27.402031
75%           0.015246      73.422478
max           0.110852     522.031250
======304137=([0.17975176])======
       position_errors  ang_errs(sum)
count        20.000000      20.000000
mean          0.011594      56.769429
std           0.013596     106.831039
min           0.003089       0.596576
25%           0.006001   

In [ ]:
path_following(robot=panda, Path_dir=Path_dir, model=nflow, knn=knn, F=F)

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
target_poses = load_numpy(file_path=traj_dir + 'ee_traj.npy')
# traj_dir, target_poses = get_target_poses(robot=panda)
exp_dir = traj_dir + f'exp_5_single_obj/'
data = {fn: [] for fn in file_names}

# -> unrefined solutions
mean_position_errors = np.zeros((num_ikflow_trails,))
mean_ang_errs = np.zeros((num_ikflow_trails,))
for ikflow_i in range(num_ikflow_trails):
    solutions, position_errors, _, _, _, _ = ik_solver.solve_n_poses(
        target_poses, 
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    df = eval_J_traj(robot=panda, J_traj=solutions.to('cpu'), P_path=target_poses, position_errors=position_errors)
    mean_position_errors[ikflow_i], mean_ang_errs[ikflow_i]  = df.mean().values
data['ikflow_l2'] = mean_position_errors
data['ikflow_ang'] = mean_ang_errs    


In [ ]:
df = pd.DataFrame(mean_position_errors)
df.describe()

In [ ]:
df = pd.DataFrame(mean_ang_errs)
df.describe()